# S&P 500 Analysis

Author: Xavien Aguigui

Course Project, UC Irvine, Math 10, W22


## Introduction
In this project, I want to analyze the effect of the market cap on revenue growth from the S&P 500 Stocks dataset. Since we're dealing with a smaller dataset, I will use linear regression to analyze my results to conclude if there is a correlation between the variables being measured. 

## Main portion of the project
Dataset: [S&P 500 Stocks from Kaggle](https://www.kaggle.com/andrewmvd/sp-500-stocks)

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler



### Data Cleaning
Originally, the two columns (Market Cap and Revenue Growth) had null values with some outliers that affected the Linear Regression analysis. The following code will clean the dataset so that I can perform my Linear Regression analysis will (hopefully) minimal error. 

In [2]:
df = pd.read_csv("sp500_companies.csv", na_values = "")
df.dropna(inplace = True) # dropping all of the null values from the original list
                          # so that we can actually analyze the data
                          # noticed a couple of outliers in the data
                          # so further cleaning is necessary

In [3]:
df.describe() # need to analyze the difference in scaling between Marketcap
# and revenue growth to see what possible cleaning we can do

,Currentprice,Marketcap,Ebitda,Revenuegrowth,Fulltimeemployees,Weight
count,427.000000,4.270000e+02,4.270000e+02,427.000000,4.270000e+02,427.000000
mean,194.694649,8.040980e+10,6.080097e+09,0.401581,5.432068e+04,0.002080
std,353.884696,2.193703e+11,1.320488e+10,1.976526,1.512113e+05,0.005673
min,9.950000,4.852946e+09,-4.127000e+09,-0.198000,1.630000e+02,0.000126
25%,60.945000,1.628580e+10,1.269000e+09,0.073500,9.165000e+03,0.000421
50%,116.110000,3.047182e+10,2.519000e+09,0.128000,1.897500e+04,0.000788
75%,209.125000,6.299591e+10,5.303000e+09,0.250500,5.200000e+04,0.001629
max,4648.430000,2.569326e+12,1.282180e+11,30.650000,2.300000e+06,0.066447


In [4]:
# need to eliminate the outliers by calling the IQR to inspect the 
# elements
cols = ['Marketcap','Revenuegrowth']
Q1  = df[cols].quantile(0.25)
Q3 = df[cols].quantile(0.75)
IQR = Q3 - Q1

IQR



Marketcap        4.671011e+10
Revenuegrowth    1.770000e-01
dtype: float64

In [5]:
# cleaning the dataset by removing the outliers
df_clean = df[~((df[cols] < (Q1-1.5 *IQR)) | (df[cols]>(Q3 + 1.5 * IQR))).any(axis=1)]


In [6]:
df_clean 

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
61,NMS,AMGN,Amgen Inc.,Amgen Inc.,Healthcare,Drug Manufacturers—General,231.10,128729407488,1.258200e+10,0.032,Thousand Oaks,CA,United States,24200.0,"Amgen Inc. discovers, develops, manufactures, ...",0.003329
62,NMS,HON,Honeywell International Inc.,Honeywell International Inc.,Industrials,Conglomerates,183.24,126146633728,8.730000e+09,0.087,Charlotte,NC,United States,103000.0,Honeywell International Inc. operates as a div...,0.003262
64,NYQ,LMT,Lockheed Martin Corporation,Lockheed Martin Corporation,Industrials,Aerospace & Defense,448.99,123825143808,9.010000e+09,0.041,Bethesda,MD,United States,114000.0,"Lockheed Martin Corporation, a security and ae...",0.003202
67,NYQ,IBM,International Business Machines,International Business Machines Corporation,Technology,Information Technology Services,125.68,112649494528,1.565900e+10,0.003,Armonk,NY,United States,345900.0,International Business Machines Corporation pr...,0.002913
68,NYQ,DE,Deere & Company,Deere & Company,Industrials,Farm & Heavy Construction Machinery,364.93,112182034432,9.040000e+09,0.160,Moline,IL,United States,75550.0,Deere & Company manufactures and distributes v...,0.002901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,NYQ,LW,"Lamb Weston Holdings, Inc.","Lamb Weston Holdings, Inc.",Consumer Defensive,Packaged Foods,51.76,7560479232,5.891000e+08,0.129,Eagle,ID,United States,7800.0,"Lamb Weston Holdings, Inc. produces, distribut...",0.000196
491,NYQ,RL,Ralph Lauren Corporation,Ralph Lauren Corporation,Consumer Cyclical,Apparel Manufacturing,101.01,7436053504,1.043900e+09,0.267,New York,NY,United States,12100.0,"Ralph Lauren Corporation designs, markets, and...",0.000192
496,NYQ,NLSN,Nielsen N.V.,Nielsen Holdings plc,Industrials,Consulting Services,17.28,6202258944,1.034000e+09,0.055,New York,NY,United States,43000.0,"Nielsen Holdings plc, together with its subsid...",0.000160
497,NMS,IPGP,IPG Photonics Corporation,IPG Photonics Corporation,Technology,Semiconductor Equipment & Materials,112.14,5978059776,4.410270e+08,0.191,Oxford,MA,United States,6060.0,IPG Photonics Corporation develops and manufac...,0.000155


In [7]:
df_clean.shape

(331, 16)

### Machine Learning (Linear Regression)

In this portion of the project, I standardized my Market cap from the S&P 500 stock dataset due to the drastic difference in scaling when compared to Revenue Growth. Additionally, the scaling is a bit awkward since I originally intended to measure Marketcap vs. Revenue. 

In [25]:
scl = StandardScaler().fit(df_clean[["Marketcap"]])
df_clean['MC_scaled'] = scl.transform(df_clean[["Marketcap"]])

# scaling our X value of Market Cap so that it is easier to interpret our results
# Also, making a new column in the orginal Dataframe with the new scaled values of 
# Marketcap

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
X_train,X_test,y_train,y_test = train_test_split(df_clean[["MC_scaled"]],df_clean["Revenuegrowth"], test_size = 0.2)
# splitting dataset into training/testing sets 
# with the training set holding 80% of the data and testing the remaining 20%
# this is used to see if our regression line is accurate in predicting 
# the true values of our original dataset



In [10]:
reg = LinearRegression()
reg.fit(X_train, y_train) 
# need to train the model after we have rescaled the data 
# in order to make accurate predictions on the trend of the variables measured

LinearRegression()

In [11]:
y_pred = reg.predict(X_test)
y_pred 
# the following show predicted numerical values from the testing set
# based on the training model

array([0.13683486, 0.13500788, 0.13585697, 0.13812171, 0.12214518,
       0.12161375, 0.13668093, 0.13432147, 0.13707596, 0.1346636 ,
       0.13782354, 0.13573909, 0.12866054, 0.13809498, 0.12674453,
       0.13757657, 0.13225075, 0.13762232, 0.13293582, 0.13364909,
       0.13205397, 0.13790633, 0.13750192, 0.13141033, 0.13100113,
       0.13573654, 0.13830739, 0.13743907, 0.13832281, 0.12295321,
       0.13782726, 0.13294912, 0.13825819, 0.136615  , 0.13574919,
       0.13532617, 0.12999101, 0.13499649, 0.13724303, 0.13395602,
       0.13635738, 0.13474462, 0.1367551 , 0.13693491, 0.13870593,
       0.13440439, 0.13382084, 0.1378882 , 0.12922038, 0.13382983,
       0.12856345, 0.13116663, 0.13570368, 0.1262327 , 0.13198489,
       0.13432154, 0.13540461, 0.13465664, 0.13777811, 0.13576223,
       0.13762969, 0.12833533, 0.12600606, 0.13155366, 0.13284938,
       0.13583129, 0.13594435])

In [12]:
y_test 
# calling y_test to see how close our predicted 
# values are to the true values of the testing set

379   -0.004
277    0.145
315    0.370
473    0.033
81     0.233
       ...  
101    0.082
161    0.309
199    0.132
313    0.149
319    0.056
Name: Revenuegrowth, Length: 67, dtype: float64

In [13]:
reg.score(X_train,y_train) 
# this low regression score indicates that the performance of the regressor is fairly poor
# which can stem from the little correlation between the variables measured

0.001689231764561816

In [14]:
reg.score(X_test,y_test) 
# since both my test and training scores are significantly low 
# this may indicate underfitting of the model

0.005110495477865906

In [15]:
reg.coef_ 
# indicating that there might 
# be a negative relation between the variables measured

array([-0.00478578])

In [17]:
test_error = mean_absolute_error(reg.predict(X_test), y_test) # measures the magnitude of error in the testing set
test_error

0.07971111826098215

In [18]:
train_error = mean_absolute_error(reg.predict(X_train), y_train) # same for the training set
train_error # very close for both training and testing sets 
# however, since the testing error is lower than the train error
# this means that there is sampling bias in our analysis



0.09206105708734641

### Data Visualizations
For this part of the project, I've modeled Market Cap vs. Revenue growth along with the regression line in both graphs. The scaling for Marketcap is awkward to read at first, but it's meant to make the scatterplot a little easier to read with the regression line.
Additionally, I've made a couple of cosmetic adjustments to the graph to make it easier to understand with proper labels and scaling.

In [20]:
c1 = alt.Chart(df_clean).mark_circle().encode(
    x = alt.X("Marketcap",axis = alt.Axis(format='$', title = 'Market Cap')), 
    y = alt.Y("Revenuegrowth",axis = alt.Axis(format ='%', title = 'Revenue Growth')),   
    color = "Shortname",
).properties(
    title = "Market cap vs. Revenue growth",
    width = 700,                
    height = 400,
)

In [21]:
c1 + c1.transform_regression('Marketcap','Revenuegrowth').mark_line()
# combining the altair chart with the regression line 

alt.LayerChart(...)

In [22]:
X = df_clean.MC_scaled 
x_range = np.linspace(X.min(), X.max())
y_range = reg.predict(x_range.reshape(-1,1))
fig = px.scatter(df_clean, x = 'MC_scaled', y = 'Revenuegrowth', color = "Shortname",
    labels = {
            "MC_scaled": "Market Cap",
            "Revenuegrowth": "Revenue Growth",
            "Shortname": "Company Name"
            },
    title = "Correlation Between Market Capitalization vs. Revenue Growth")
fig.add_traces(go.Scatter(x = x_range, y= y_range, name = 'Regression Fit'))
fig.show()

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


## Summary
After analyzing the data, the results were inconclusive as other financial indicators were not taken into account (cash flow, profitability, etc.) and an underfitting of the model. The reason behind this stems from measuring the wrong variable of Revenue growth which should instead be Revenue. In turn, the predictions from the testing/training set were not accurate. 

## References
I plan on using references such as tutorials from youtube or even articles that will help me with the syntax for plotly or adjust my data accordingly so that I can have accurate results. 

Some References:
Some References:
[Python Treatment for Outliers (Analytics Vidhya)](https://www.analyticsvidhya.com/blog/2021/03/zooming-out-a-look-at-outlier-and-how-to-deal-with-them-indata-science/)

[Cleaning Dataset of Outliers (StackOverflow)](https://stackoverflow.com/questions/35827863/remove-outliers-in-pandas-dataframe-using-percentiles)

[Scatterplot with Regression Line using Altair (datavizpyr)](https://datavizpyr.com/scatter-plot-with-regression-line-using-altair-in-python/)

[Setting the Font, Title, Legend Entries, and Axis Titles in Python](https://plotly.com/python/figure-labels/)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a9778175-04d7-4fdb-b002-4db6c2eb45d1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>